In [8]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [9]:
x = sp.symbols('x')
f_x = x**2 + 2*sp.sin(x)

In [10]:
c_x= sp.lambdify(x, f_x)
f_x_dot = sp.diff(f_x, x)
c_x_dot = sp.lambdify(x, f_x_dot)
f_x_ddot = sp.diff(f_x_dot, x)
c_x_ddot = sp.lambdify(x, f_x_ddot)

In [12]:
def taylor_expansion(xk, c_x, c_x_dot, c_x_ddot, xk_range):
    return c_x(xk) + c_x_dot(xk) * (xk_range - xk) + 0.5 * c_x_ddot(xk) * (xk_range - xk)**2

def newton_optimizor_1D(init_x, c_x, c_x_dot, c_x_ddot, tol=1e-8, max_iter=1000):
    x = init_x
    all_x = []
    all_c = []
    all_x.append(x)
    all_c.append(c_x(x))
    for i in range(max_iter):
        if c_x(x) <= tol:
            print(f"Solution converge after {i+1} iterations. Minimum:={c_x(x)} at x:={x}")
            return x, c_x(x), all_x, all_c
        x -= c_x_dot(x) / c_x_ddot(x)

        all_x.append(x)
        all_c.append(c_x(x))
    print(f"Solution did not converge. Maximum iterations reached.")
    return None
    
# Newton optimization
x0 = 6
x_optim, c_x_optim, all_x, all_c = newton_optimizor_1D(init_x=x0, c_x=c_x, c_x_dot=c_x_dot, c_x_ddot=c_x_ddot)

Solution converge after 4 iterations. Minimum:=-0.7026462785465547 at x:=-0.49191148608506574


In [13]:
# Plot    
def update_plot(frame):
    xk = all_x[frame]
    cost = all_c[frame]
    t_xk = taylor_expansion(xk=xk, c_x=c_x, c_x_dot=c_x_dot, c_x_ddot=c_x_ddot, xk_range=x_values)

    sc1.set_offsets(np.column_stack((all_x[:frame], all_c[:frame])))
    if frame == len(all_x) - 1:
        sc2.set_offsets(np.array([[xk, cost]]))
    taylor_line.set_data(x_values, t_xk)

# Set up the initial plot
fig, ax = plt.subplots()
x_values = np.linspace(-5, 5, 100)
line, = ax.plot(x_values, c_x(x_values), label='Objective Function')
sc1 = ax.scatter([], [], marker='x', s=50, color='red', label='Iteration Points')
sc2 = ax.scatter([], [], marker='x', s=200, color='black', label='Minimum')
taylor_line, = ax.plot([], [], linestyle='--', label='Taylor Expansion')

ax.set_xlim(-10, 10)
ax.set_ylim(-20, 50)
ax.set_xlabel('x')
ax.set_ylabel('c_x')
ax.set_title('Newton Optimization 1D')
ax.legend(loc='lower left')
ax.grid(True)


# Create animation
anim = FuncAnimation(fig, update_plot, frames=len(all_x),repeat=True)
plt.close()  # to avoid showing last plot separately
HTML(anim.to_jshtml())
